# 2. Model Training (student indicator)

## 1. Import Data and required packages

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
# from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [9]:
df = pd.read_csv('data/students.csv')
print(df.shape)
df.head()

(1000, 8)


,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


## 2. Prepare X and y variables 
- Here, let's predict the math score
- Then, our independent features(X) are other columns except the math score.


In [14]:
X = df.drop(columns=['math_score'], axis=1, inplace=False) #return the copy 
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [16]:
y = df['math_score']
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

## 3. Check Numerical, Categorical features
- Since categorical values have small number of them, we can perform one-hot encoding => converting to numerical values.
- For numerical values, we apply standardization.

### 3.1 Data types in X 

In [18]:
X.dtypes

gender                         object
race_ethnicity                 object
parental_level_of_education    object
lunch                          object
test_preparation_course        object
reading_score                   int64
writing_score                   int64
dtype: object

In [17]:
#excluding numerical values (scores columns)
print('The unique categories in \'gender\' are : ', df['gender'].unique())
print('The unique categories in \'race_ethnicity\' are : ', df['race_ethnicity'].unique())
print('The unique categories in \'parental_level_of_education\' are : ', df['parental_level_of_education'].unique())
print('The unique categories in \'lunch\' are : ', df['lunch'].unique())
print('The unique categories in \'test_preparation_course\' are : ', df['test_preparation_course'].unique())

The unique categories in 'gender' are :  ['female' 'male']
The unique categories in 'race_ethnicity' are :  ['group B' 'group C' 'group A' 'group D' 'group E']
The unique categories in 'parental_level_of_education' are :  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
The unique categories in 'lunch' are :  ['standard' 'free/reduced']
The unique categories in 'test_preparation_course' are :  ['none' 'completed']


### 3.2 Check each features

In [28]:
numerical_features = X.select_dtypes(exclude='object').columns
categorical_features = X.select_dtypes(include='object').columns

print("Numerical features are : ", numerical_features)
print("Categorical featrues are : ", categorical_features)


Numerical features are :  Index(['reading_score', 'writing_score'], dtype='object')
Categorical featrues are :  Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')


## 4. Construct pipelines for pre-processing
- Create a column transformers to construct a pipeline
- OneHotEncoder (categorical value => numerical value)
- StandardScaler (numerical value => standardization)

In [29]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler # these two 
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

#this processor can be used later to fit, transform, etc.
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, categorical_features),
        ("StandardScaler", numeric_transformer, numerical_features),
        
    ]
)

In [30]:
X = preprocessor.fit_transform(X)
X.shape

(1000, 19)

In [32]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

## 5. Split the dataset

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

##  6. Evaluate Functions returning different metrics after model training


In [33]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    
    return mae, rmse, r2_square 

## 7. Train models and evaluate

In [41]:
models = {
    'Linear Regression' : LinearRegression(),
    'Lasso' : Lasso(),
    'Ridge' : Ridge(),
    'K-Neighbors Regressor' : KNeighborsRegressor(),
    'Decision Tree' : DecisionTreeRegressor(),
    'Random Forest Regressor' : RandomForestRegressor(),
    'XGBRegressor' : XGBRegressor(),
    'AdaBoost Regressor' : AdaBoostRegressor(),
}

model_list = []
r2_list = []

for k,v in models.items():
    model = v

    #train model
    model.fit(X_train, y_train)
    
    #predict
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    #evaluate train and test dataset
    train_mae, train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae, test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)
    
    ### print the result
    print(k)
    model_list.append(k)
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(train_mae))
    print("- R2 Score: {:.4f}".format(train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(test_mae))
    print("- R2 Score: {:.4f}".format(test_r2))
    r2_list.append(test_r2)
    
    print('='*35)
    print('\n')
    

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3253
- Mean Absolute Error: 4.2673
- R2 Score: 0.8742
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3907
- Mean Absolute Error: 4.2078
- R2 Score: 0.8806


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5938
- Mean Absolute Error: 5.2063
- R2 Score: 0.8071
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.5197
- Mean Absolute Error: 5.1579
- R2 Score: 0.8253


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.3233
- Mean Absolute Error: 4.2650
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3904
- Mean Absolute Error: 4.2111
- R2 Score: 0.8806


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.7033
- Mean Absolute Error: 4.5105
- R2 Score: 0.8557
-----------------------

In [45]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(
    by=['R2_Score'], ascending=False)

,Model Name,R2_Score
2,Ridge,0.880593
0,Linear Regression,0.880578
5,Random Forest Regressor,0.853193
7,AdaBoost Regressor,0.847854
1,Lasso,0.825320
6,XGBRegressor,0.821589
3,K-Neighbors Regressor,0.783902
4,Decision Tree,0.725978
